In [2]:
from pathlib import Path
import pydicom
import numpy as np
import SimpleITK as sitk
from skimage.draw import polygon  


def voxelise_rtstruct(rt_path, ct_dir, out_dir, z_offset=0, new_z_spacing=1.0):

    reader = sitk.ImageSeriesReader()
    series_files = reader.GetGDCMSeriesFileNames(str(ct_dir))
    reader.SetFileNames(series_files)
    ct_img = reader.Execute()

    spacing = np.array(ct_img.GetSpacing())       
    origin = np.array(ct_img.GetOrigin())          
    direction = np.array(ct_img.GetDirection())    
    size = np.array(ct_img.GetSize())[::-1]        
    z_positions = np.arange(size[0]) * spacing[2] + origin[2]

    rt = pydicom.dcmread(str(rt_path))
    print(f"Processing {Path(rt_path).name}...")

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    mha_dir = out_dir / "mha"
    nii_dir = out_dir / "nii"
    mha_dir.mkdir(exist_ok=True)
    nii_dir.mkdir(exist_ok=True)

    for roi, roi_contour in zip(rt.StructureSetROISequence, rt.ROIContourSequence):
        name = roi.ROIName.replace(" ", "_")
        mask = np.zeros(size, dtype=bool)
        for contour in roi_contour.ContourSequence:
            pts = np.array(contour.ContourData).reshape(-1, 3)
            z = np.mean(pts[:, 2])
            slice_idx = np.argmin(np.abs(z_positions - z)) + z_offset
            slice_idx = np.clip(slice_idx, 0, mask.shape[0] - 1)

            R = direction.reshape(3, 3)
            ijk = ((pts - origin) / spacing) @ np.linalg.inv(R).T
            x, y = ijk[:, 0], ijk[:, 1]

            rr, cc = polygon(y, x, shape=mask.shape[1:])
            mask[slice_idx, rr, cc] = True

        mask_img = sitk.GetImageFromArray(mask.astype(np.uint8))
        mask_img.CopyInformation(ct_img)

        mha_path = mha_dir / f"{name}.mha"
        sitk.WriteImage(mask_img, str(mha_path))
        print(f"  → saved {mha_path}")

        nii_path = nii_dir / f"{name}.nii.gz"
        sitk.WriteImage(mask_img, str(nii_path))
        print(f"  → saved {nii_path}")

    print("Done.\n")


rt_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\RS_file\RS1.2.752.243.1.1.20230830173944533.9000.76237.dcm")
ct_dir  = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\full_CT")
out_dir = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations")

voxelise_rtstruct(rt_path, ct_dir, out_dir)


Processing RS1.2.752.243.1.1.20230830173944533.9000.76237.dcm...
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\mha\Dentition.mha
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Dentition.nii.gz
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\mha\Orbit_R.mha
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Orbit_R.nii.gz
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\mha\Orbit_L.mha
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Orbit_L.nii.gz
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\mha\Nasal_bone.mha
  → saved Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manu

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load("/home/abigail/mphys/1Zr/CT_WHOLE_CNS_cropped.nii")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    "head": "/home/abigail/mphys/1Zr/head.nii.gz",
    "mandible": "/home/abigail/mphys/1Zr/mandible.nii.gz",
    "sinus_frontal": "/home/abigail/mphys/1Zr/sinus_frontal.nii.gz",
    "sinus_maxillary": "/home/abigail/mphys/1Zr/sinus_maxillary.nii.gz",
    "skull": "/home/abigail/mphys/1Zr/skull.nii.gz",
    "teeth_upper": "/home/abigail/mphys/1Zr/teeth_upper.nii.gz",
    "teeth_lower": "/home/abigail/mphys/1Zr/teeth_lower.nii.gz",
}

segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    "head": (0.2, 0.4, 1.0, 0.25),          # blueish
    "mandible": (1.0, 0.6, 0.0, 0.35),      # orange
    "sinus_frontal": (0.2, 1.0, 0.2, 0.35), # green
    "sinus_maxillary": (0.6, 0.2, 1.0, 0.35), # purple
    "skull": (0.9, 0.9, 0.8, 0.25),         # bone
    "teeth_upper": (1.0, 0.2, 0.2, 0.35),   # red
    "teeth_lower": (1.0, 0.6, 0.4, 0.35),   # light red/orange
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x9x011Zr"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)
